In [1]:
import pandas as pd

In [2]:
#1. 

user = "root"
password = "Admin149."
host = "127.0.0.1"
port = "3306"
db = "sakila"
from sqlalchemy import create_engine, text
engine = create_engine(f'mysql+mysqlconnector://{user}:{password}@{host}:{port}/{db}')

In [3]:
#2.

def rentals_month(engine, month, year):
    # SQL query to retrieve rental data for the specified month and year
    query = text("""
        SELECT *
        FROM rental
        WHERE MONTH(rental_date) = :month AND YEAR(rental_date) = :year
    """)
    
    # Execute the query and fetch the results into a DataFrame
    with engine.connect() as connection:
        result = connection.execute(query, month=month, year=year)
        df = pd.DataFrame(result.fetchall(), columns=result.keys())
    
    return df

In [5]:
#3.

def rental_count_month(rental_df, month, year):
    # Group the DataFrame by customer_id and count the number of rentals for each customer
    rental_counts = rental_df.groupby('customer_id').size().reset_index(name=f'rentals_{month:02d}_{year}')
    
    return rental_counts

In [6]:
#4.

def compare_rentals(df1, df2):
    # Merge the two DataFrames on 'customer_id'
    combined_df = pd.merge(df1, df2, on='customer_id', how='outer', suffixes=('_1', '_2'))
    
    # Fill NaN values with 0 (if a customer didn't rent in one of the months)
    combined_df.fillna(0, inplace=True)
    
    # Calculate the difference between the rental counts
    combined_df['difference'] = combined_df.iloc[:, 1] - combined_df.iloc[:, 2]
    
    return combined_df